In [81]:
from pprint import pprint

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt


from prepare import basic_clean, tokenize, remove_stopwords, stem, lemmatize

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

import re

import nltk
import unicodedata

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from env import get_db_url

In [6]:
url = get_db_url("spam_db")
sql = "SELECT * FROM spam"

df = pd.read_sql(sql, url, index_col="id")
df.head()

X = df.text
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2)

tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

lm = LogisticRegression().fit(X_train, y_train)

train['predicted'] = lm.predict(X_train)
test['predicted'] = lm.predict(X_test)

In [38]:
X

id
0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: text, Length: 5572, dtype: object

In [7]:
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.predicted, train.actual))
print('---')
print(classification_report(train.actual, train.predicted))


Accuracy: 97.49%
---
Confusion Matrix
actual      ham  spam
predicted            
ham        3858   111
spam          1   487
---
              precision    recall  f1-score   support

         ham       0.97      1.00      0.99      3859
        spam       1.00      0.81      0.90       598

    accuracy                           0.97      4457
   macro avg       0.98      0.91      0.94      4457
weighted avg       0.98      0.97      0.97      4457



In [8]:
print('Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(test.predicted, test.actual))
print('---')
print(classification_report(test.actual, test.predicted))


Accuracy: 97.13%
---
Confusion Matrix
actual     ham  spam
predicted           
ham        965    31
spam         1   118
---
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98       966
        spam       0.99      0.79      0.88       149

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.93      1115
weighted avg       0.97      0.97      0.97      1115



In [57]:
df.head()

,label,text
id,,
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [74]:
# Cleans the datasets for spam/ham
words_list = []

for index, row in df.iterrows():
    words = basic_clean(row.text)# pulling only the column row.text
#     words_dict = {
#         row.label : words
#     }
    words_list.append(words)
    

In [76]:
df['text'] = pd.Series(words_list)

In [83]:
# Applies all the finctions to the strings or words in the column
df['text'] = df['text'].apply(lambda string: remove_stopwords(tokenize(basic_clean(string))))
df.head()

,label,text
id,,
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah ' think goes usf lives around though


In [49]:
ADDITIONAL_STOPWORDS = ['r', 'u', '2', 'ltgt']

def clean(text):
    'A simple function to cleanup text data'
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    text = (unicodedata.normalize('NFKD', text)
             .encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [51]:
def split_data():


    # split test off, 20% of original df size. 
    train_validate, test = train_test_split(df, test_size=.2, 
                                            random_state=42)

    # split validate off, 30% of what remains (24% of original df size)
    # thus train will be 56% of original df size. 
    train, validate = train_test_split(train_validate, test_size=.3, 
                                       random_state=42)
    
    return train, validate, test

In [55]:
def X_y_split():
    
    train, validate, test = split_data()

    # split train into X (dataframe, drop target) & y (series, keep target only)
    X_train = train.drop(columns=['text'])
    y_train = train['text']
    # split validate into X (dataframe, drop target) & y (series, keep target only)
    X_validate = validate.drop(columns=['text'])
    y_validate = validate['text']
    # split test into X (dataframe, drop target) & y (series, keep target only)
    X_test = test.drop(columns=['text'])
    y_test = test['text']
    
    
    return X_train, y_train, X_validate, y_validate, X_test, y_test

In [84]:
# Applies all the finctions to the strings or words in the column
df['text'] = df['text'].apply(lambda string: tokenize(string))
df.head()

,label,text
id,,
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah ' think goes usf lives around though


In [50]:
# Creates a list of all the words in ham, spam and all the words
ham_words = clean(' '.join(df[df.label == 'ham'].text))
spam_words = clean(' '.join(df[df.label == 'spam'].text))
all_words = clean(' '.join(df.text))

In [52]:
train, validate, test = split_data()

In [56]:
X_train, y_train, X_validate, y_validate, X_test, y_test = X_y_split()